# 3. Creating a Simple Agent

Welcome to the world of AI Agents! This notebook introduces the concept of agents and how to build a very basic one using the `google-genai` Python SDK for Vertex AI.

**What is an Agent?**
An agent is an AI system that can:
1.  **Understand** a user's goal.
2.  **Plan** the steps needed to achieve it.
3.  **Use Tools** (like functions, APIs, or other models) to execute those steps.
4.  **Reason** about the results and adjust its plan.

Think of it like an assistant that doesn't just answer questions based on its training data, but can actively *do* things.

**Our Goal:** Build a simple agent that can use a basic calculator tool.

## Setup

First, let's import the necessary libraries and configure the Vertex AI client, just like in the previous notebooks.

In [ ]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv

print(f"google-genai SDK imported successfully.")

In [ ]:
# Load environment variables (if you have a .env file)
load_dotenv()

# --- TODO: Configure these values for your environment --- 
# Ensure PROJECT_ID is set in your environment or replace '...'
PROJECT_ID = os.getenv("PROJECT_ID", "...") 
LOCATION = "us-central1"   # Or your preferred Vertex AI region
# -----------------------------------------------------

# Attempt to initialize the client
print(f"Configuring client for Project: {PROJECT_ID}, Location: {LOCATION}")
try:
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
    print("✅ google-genai client configured for Vertex AI.")
except Exception as e:
    print(f"❌ Error configuring client: {e}")
    print("Please ensure your PROJECT_ID is correct and you have authenticated via 'gcloud auth application-default login'.")

## Step 1: Define a Tool

Agents need tools to interact with the outside world or perform specific tasks. We'll define a simple calculator function.

Crucially, we also need to provide a *description* of the tool in a specific format (FunctionDeclaration) so the LLM knows *when* and *how* to use it.

In [ ]:
# The actual Python function
def simple_calculator(operand1: float, operator: str, operand2: float) -> float:
    """Performs a basic arithmetic operation.
    
    Args:
        operand1: The first number.
        operator: The operation ('+', '-', '*', '/').
        operand2: The second number.
        
    Returns:
        The result of the calculation.
    """
    print(f"--> Tool Called: simple_calculator({operand1}, '{operator}', {operand2})")
    if operator == '+':
        return operand1 + operand2
    elif operator == '-':
        return operand1 - operand2
    elif operator == '*':
        return operand1 * operand2
    elif operator == '/':
        if operand2 == 0:
            return "Error: Division by zero"
        return operand1 / operand2
    else:
        return f"Error: Unknown operator '{operator}'"

# Describe the tool for the LLM
calculator_tool = types.FunctionDeclaration(
    name="simple_calculator",
    description="Performs basic arithmetic calculations: addition, subtraction, multiplication, division.",
    parameters=types.Schema(
        type=types.Type.OBJECT,
        properties={
            'operand1': types.Schema(type=types.Type.NUMBER, description="First number in the calculation"),
            'operator': types.Schema(type=types.Type.STRING, description="The arithmetic operator (+, -, *, /)", enum=['+', '-', '*', '/']),
            'operand2': types.Schema(type=types.Type.NUMBER, description="Second number in the calculation")
        },
        required=['operand1', 'operator', 'operand2']
    )
)

print("✅ Calculator tool and its description defined.")

## Step 2: Create the Agent (Model)

Now, we select a generative model that supports function calling (most Gemini models do) and tell it about the tools it can use.

In [ ]:
# --- TODO: Choose a suitable Gemini model --- 
# Models like 'gemini-1.5-flash-001' or 'gemini-1.0-pro' usually work well.
AGENT_MODEL_NAME = "gemini-1.5-flash-001" 
# ------------------------------------------

try:
    # Get the model instance
    agent_model = client.models.get_model(f"models/{AGENT_MODEL_NAME}")
    print(f"✅ Using agent model: {AGENT_MODEL_NAME}")
except Exception as e:
    print(f"❌ Error getting model: {e}")
    agent_model = None # Ensure model is None if setup failed

## Step 3: Interact with the Agent

Let's ask the agent a question that requires calculation. The agent should:
1.  Recognize that calculation is needed.
2.  Identify the `simple_calculator` tool as appropriate.
3.  Generate a `FunctionCall` asking us (the caller) to execute the tool with the correct arguments.
4.  Receive the result back from us (as a `FunctionResponse`).
5.  Use the result to formulate the final answer.

In [ ]:
if agent_model: # Only proceed if the model was loaded successfully
    # --- TODO: Ask a question that requires calculation --- 
    user_query = "What is 123 times 4?"
    # ----------------------------------------------------
    
    print(f"
User Query: {user_query}")
    
    # 1. First call to the model - it should request the tool
    print("
--- Sending query to agent ---")
    response = agent_model.generate_content(
        contents=user_query,
        tools=[types.Tool(function_declarations=[calculator_tool])] # Tell the model about the tool
    )
    
    # Check if the model wants to call our function
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        print(f"
--- Agent wants to call function: {function_call.name} ---")
        print(f"Arguments: {dict(function_call.args)}")
        
        # 2. Execute the function the agent requested
        if function_call.name == "simple_calculator":
            # Extract arguments (handle potential errors later in real apps)
            args = dict(function_call.args)
            op1 = args.get('operand1')
            op = args.get('operator')
            op2 = args.get('operand2')
            
            # Call the actual Python function
            tool_result = simple_calculator(operand1=op1, operator=op, operand2=op2)
            print(f"Tool Result: {tool_result}")
            
            # 3. Send the result back to the model
            print("
--- Sending tool result back to agent ---")
            response_with_result = agent_model.generate_content(
                # Provide the previous conversation history AND the tool result
                contents=[
                    response.candidates[0].content, # The agent's previous turn (the function call)
                    types.Part(function_response=types.FunctionResponse(
                        name=function_call.name,
                        response={'result': tool_result} # The result from our function
                    ))
                ],
                tools=[types.Tool(function_declarations=[calculator_tool])] # Still need to provide tool info
            )
            
            # 4. Get the final answer
            final_answer = response_with_result.text
            print(f"
--- Agent's Final Answer ---")
            print(final_answer)
            
        else:
            print(f"Error: Agent requested unknown function '{function_call.name}'")
            
    else:
        # The model answered directly without using the tool
        print("
--- Agent answered directly ---")
        print(response.text)
else:
    print("
Skipping interaction because agent model setup failed.")

## Recap & Next Steps

You've just built and interacted with a basic agent!

**Key Concepts:**
*   **Tools:** Functions the agent can use.
*   **FunctionDeclaration:** Describing tools for the LLM.
*   **FunctionCall:** The LLM requesting a tool execution.
*   **FunctionResponse:** Providing the tool's result back to the LLM.
*   **Multi-turn Conversation:** The interaction often involves back-and-forth between the user, LLM, and tools.

> #### 🎁 Bonus exercises 📝
> - **Add More Tools:** Create another tool (e.g., one that returns the current date) and add its `FunctionDeclaration` to the `tools` list.
> - **Error Handling:** What happens if the user asks for division by zero? How could you make the agent handle tool errors more gracefully?
> - **Complex Queries:** Try asking multi-step questions like "What is 5 plus 3, and then multiply the result by 2?". Does the agent handle it in one go or multiple steps?
> - **Explore Frameworks:** Libraries like LangChain or LlamaIndex provide higher-level abstractions for building more complex agents.